# Segmenting and Clustering Neighborhoods in Toronto

## 1. Introduction

This notebook will use the Foursquare API to explore neighborhoods in Toronto. The k-means clustering algorithm is used to complete this task and the Folium library to visualize the neighborhoods in Toronto and their emerging clusters.

## 2. Download the dependencies

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


## 3. Download and Explore the Dataset

<p> Since the dataset for Toronto is not readily available, it is required to download and wrangle the dataset from wikipedia </p>

### 3.1 Read content of the wikipedia page containing the data about Toronto

In [3]:
# Get data from the wikipedia page
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
toronto_data_wiki=requests.get(url)
# Convert the data to string and display the data
toronto_wiki_page=toronto_data_wiki.text

### 3.2 Wrangle the data

<p> The data read from the wikipedia page contains a table which needs to be converted to a pandas dataframe. This can be done with the help of the BeautifulSoup package </p>

In [4]:
# Install beautiful soup
!conda install -c anaconda beautifulsoup4 --yes

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - beautifulsoup4


The following packages will be UPDATED:

    cryptography:    2.3.1-py36hdffb7b8_0    conda-forge --> 2.4.1-py36h1ba5d50_0    anaconda
    grpcio:          1.16.0-py36hd60e7a3_0   conda-forge --> 1.16.1-py36hf8bcb03_1   anaconda
    libarchive:      3.3.3-h823be47_0        conda-forge --> 3.3.3-h5d8350f_4        anaconda
    libcurl:         7.63.0-hbdb9355_0       conda-forge --> 7.63.0-h20c2e04_1000            
    libssh2:         1.8.0-h5b517e9_3        conda-forge --> 1.8.0-h1ba5d50_4        anaconda
    openssl:         1.0.2p-h470a237_2       conda-forge --> 1.1.1-h7b6447c_0        anaconda
    pycurl:          7.43.0.2-py36hb7f436b_0             --> 7.43.0.2-py36h1ba5d50_0         
    python:          3.6.6-h5001a0f_3        conda-forge --> 3.6.7-h0371630_0        anaconda

The following packages will be DOWNGRADED:

    ca-certificate

In [5]:
# Import the BeautifulSoup Package 
from bs4 import BeautifulSoup
bs = BeautifulSoup(toronto_wiki_page,'lxml')
# Use prettify function to determine the html table class name which needs to be extracted
#print(bs.prettify())

In [6]:
# Extract the table from the page
toronto_table = bs.find('table',{'class':'wikitable sortable'})
# toronto_table // Display the table

In [7]:
#Extract postcode, borough and neighbour hood lists based on conditions specified
rows = toronto_table.find_all('tr')
postcode=[]
borough=[]
neighbourhood=[]
for row in rows:
    cols=row.find_all('td')
    cols=[x.text.strip() for x in cols] # Cols is a list with 3 elements in the order postcode, borough,neighbourhood
    if cols:
        if(cols[1] !='Not assigned'): # Ignore cells with a borough that is Not assigned.
            borough.append(cols[1]) 
            if(cols[2] == 'Not assigned'): # If neighborhood is not assigned, it is the same as borough
                neighbourhood.append(cols[1])
            else:
                neighbourhood.append(cols[2])
            if(cols[0] != 'Not Assigned'):
                postcode.append(cols[0]) 

# Display lists
#print(postcode)
#print(borough)
#print(neighbourhood)

In [8]:
# Combine the lists to a dataframe
toronto_df_ini=pd.DataFrame(
    {'Postalcode': postcode,
     'Borough': borough,
     'Neighborhood': neighbourhood
    })
toronto_df_ini.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


<p> Concatenate the neighborhoods coming under the same postal code using lambda functions and pandas groupby function</p>

In [9]:
def concatenate_neighborhood(x):
    neigh_concat = ""
    for i in range(len(x)-1):
        neigh_concat = neigh_concat + x.iloc[i] + ", "
    neigh_concat += x.iloc[-1]
    return neigh_concat

def select_Borough(x):
    borough_sel = x.iloc[0]
    for i in range(1, len(x)):
        if borough_sel != x.iloc[i]:
            for i in x:
                print(x)
            raise Exception("Postcode comprises two Boroughs")
    return borough_sel
toronto_df = toronto_df_ini.groupby(["Postalcode"]).agg({"Borough": lambda x: select_Borough(x),
                                 "Neighborhood": lambda x: concatenate_neighborhood(x)},as_index=False)
toronto_df = toronto_df.reset_index()
toronto_df.head(5)

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## 4. Display Shape of dataframe

In [10]:
toronto_df.shape

(103, 3)

## 5. GeoSpatial Analysis

### 5.1 Download csv file

<p> Since the geocoder package is not reliable enough, the csv file provided by coursera is used for the latitude and longitude coordinated of locations </p>

In [11]:
# Download the csv file
!wget -q -O 'lat_long_data.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [12]:
# Read the csv data into a pandas data frame
lat_long_df = pd.read_csv('lat_long_data.csv')
lat_long_df.head(5)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
# Remove postal code column from lat_long_df before concatenation
lat_long_df.drop(['Postal Code'],axis=1,inplace=True)
lat_long_df.head()

,Latitude,Longitude
0,43.806686,-79.194353
1,43.784535,-79.160497
2,43.763573,-79.188711
3,43.770992,-79.216917
4,43.773136,-79.239476


In [14]:
# Concatentate the two data frames
toronto_df_geo=pd.concat([toronto_df,lat_long_df],axis=1,sort=False)
toronto_df_geo.head(5)

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### 5.2 Neighborhood Analysis in Toronto

#### 5.2.1 Print Boroughs and neighborhood information in Toronto

In [15]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_df_geo['Borough'].unique()),
        toronto_df_geo.shape[0]
    )
     )

The dataframe has 11 boroughs and 103 neighborhoods.


#### 5.2.2 Use geopy library to get the latitude and longitude values of Toronto City

In [16]:
address = 'Toronto, ON'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### 5.2.3 Create a map of Toronto with neighborhoods superimposed on top

<p> The Folium Library is used to create the map </p>

In [17]:
# create map of New York using latitude and longitude values
map_toronto= folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_geo['Latitude'], toronto_df_geo['Longitude'], toronto_df_geo['Borough'], toronto_df_geo['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='Black',
        fill=True,
        fill_color='#9400D3',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### 5.2.4 Use FourSquare API to Explore and Segment Neighborhoods in Toronto

In [18]:
# Define the foursquare credentials
CLIENT_ID = 'G4QO3DVQYFC1OQLB4RZN5LJZFZVMQJPGEZFK0GKGTHZ2A4HW' # your Foursquare ID
CLIENT_SECRET = '2HSKJYUEYRTNQUSTALPKULTNKQJMVJGCQHVK2ZZJFDN21KHV' # your Foursquare Secret
VERSION = '20190401' # Foursquare API version

#### 5.2.5 Explore a neighborhood in detail

<p> Let us explore the Downtown Toronto Neighborhood in detail. The row index I chose within Downtown Toronto is 57. This was randomly chosen </p>

In [19]:
toronto_df_geo.loc[57,'Neighborhood']

'Central Bay Street'

In [20]:
neighborhood_latitude = toronto_df_geo.loc[57, 'Latitude'] # neighborhood latitude value
neighborhood_longitude =toronto_df_geo.loc[57, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_df_geo.loc[57, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Central Bay Street are 43.6579524, -79.3873826.


#### Top 100 venues in Central Bay Street, Downtown Toronto within a radius of 500m

In [21]:
# Create the url to get the values

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

# Send the get request and view the results
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c3003e31ed219438f5c4345'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bay Street Corridor',
  'headerFullLocation': 'Bay Street Corridor, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 85,
  'suggestedBounds': {'ne': {'lat': 43.6624524045, 'lng': -79.38117421839567},
   'sw': {'lat': 43.6534523955, 'lng': -79.39359098160432}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '537d4d6d498ec171ba22e7fe',
       'name': "Jimmy's Coffee",
       'location': {'address': '82 Gerrard Street W',
        'crossStreet': 'Gerrard & LaPlante',
        'lat': 43.65842123574496,
        'lng': -79.38561319551111,
        'label

#### 5.2.7 Structure the values, clean the json and convert to a Pandas DataFrame

In [22]:
# function that extracts the category of the venue from FourSquare Lab
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [23]:
#clean the json and structure it into a pandas dataframe

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Jimmy's Coffee,Coffee Shop,43.658421,-79.385613
1,Tim Hortons,Coffee Shop,43.658570,-79.385123
2,Hailed Coffee,Coffee Shop,43.658833,-79.383684
3,The Queen and Beaver Public House,Gastropub,43.657472,-79.383524
4,Mercatto,Italian Restaurant,43.660391,-79.387664


In [24]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

85 venues were returned by Foursquare.


### 6. Explore all neighborhoods in Toronto

#### 6.1 The getnearbyvalues function is used to explore and get the values of all the neighborhoods using loops 

In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
              # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)      

#### 6.2 Create a new dataframe called toronto_venues containing the neighborhood analysis for entire Toronto City

In [26]:
toronto_venues = getNearbyVenues(names=toronto_df_geo['Neighborhood'],
                                   latitudes=toronto_df_geo['Latitude'],
                                   longitudes=toronto_df_geo['Longitude']
                                  )


Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West, Steeles West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The D

#### 6.3 Shape of Toronto Venues

In [27]:
toronto_venues.shape
toronto_venues.head()
toronto_venues = toronto_venues.reset_index()

#### 6.5 Count of Venues for each neighborhood

In [28]:
toronto_venues.groupby('Neighborhood').count()

,index,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100,100
Agincourt,5,5,5,5,5,5,5
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",3,3,3,3,3,3,3
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",13,13,13,13,13,13,13
"Alderwood, Long Branch",9,9,9,9,9,9,9
"Bathurst Manor, Downsview North, Wilson Heights",17,17,17,17,17,17,17
Bayview Village,4,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",25,25,25,25,25,25,25
Berczy Park,54,54,54,54,54,54,54


#### 6.6 Unique categories curated from all the returned venues

In [29]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 275 uniques categories.


### 7. Analyze each Neighborhood in Toronto

In [30]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
#toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.drop(['Neighborhood'],axis=1,inplace=True)
toronto_onehot.insert(0, 'Neighborhood', toronto_venues['Neighborhood'])
toronto_onehot.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

#### 7.1 Size of the one hot encoded data frame

In [31]:
toronto_onehot.shape

(2246, 275)

#### 7.2 Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [32]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.00000

#### 7.3. Confirm the new size

In [33]:
toronto_grouped.shape

(100, 275)

#### 7.4. Print each neighborhood along with the top 5 most common venues

In [34]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2           Steakhouse  0.04
3  American Restaurant  0.04
4      Thai Restaurant  0.04


----Agincourt----
                venue  freq
0              Lounge   0.2
1        Skating Rink   0.2
2      Sandwich Place   0.2
3      Breakfast Spot   0.2
4  Chinese Restaurant   0.2


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                venue  freq
0                Park  0.33
1         Coffee Shop  0.33
2          Playground  0.33
3         Yoga Studio  0.00
4  Mexican Restaurant  0.00


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                  venue  freq
0         Grocery Store  0.15
1           Pizza Place  0.15
2              Pharmacy  0.08
3  Fast Food Restaurant  0.08
4            Beer Store  0.08


----Alderwood, Long Branch----
                venue  freq
0        

#### 7.5 Create a Pandas Dataframe with the above values

In [35]:
# Sort Venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [36]:
# Create new dataframe
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,American Restaurant,Steakhouse,Thai Restaurant,Restaurant,Bakery,Asian Restaurant,Gym,Hotel
1,Agincourt,Lounge,Breakfast Spot,Sandwich Place,Chinese Restaurant,Skating Rink,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dessert Shop
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Coffee Shop,Playground,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pizza Place,Beer Store,Coffee Shop,Fried Chicken Joint,Video Store,Japanese Restaurant,Pharmacy,Fast Food Restaurant,Discount Store
4,"Alderwood, Long Branch",Pizza Place,Gym,Pharmacy,Pub,Sandwich Place,Athletics & Sports,Skating Rink,Coffee Shop,Eastern European Restaurant,Dumpling Restaurant
5,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Pizza Place,Fast Food Restaurant,Sushi Restaurant,Restaurant,Shopping Mall,Fried Chicken Joint,Frozen Yogurt Shop,Bank,Diner
6,Bayview Village,Bank,Chinese Restaurant,Japanese Restaurant,Café,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Women's Store
7,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Juice Bar,Fast Food Restaurant,Pharmacy,Restaurant,Pub,Butcher,Café,Pizza Place
8,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Cheese Shop,Farmers Market,Seafood Restaurant,Café,Italian Restaurant,Bakery,Steakhouse
9,"Birch Cliff, Cliffside West",College Stadium,Skating Rink,Café,General Entertainment,Women's Store,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant


## 8. Cluster the neighborhoods

#### 8.1 Run k-means to cluster the neighborhood into 7 clusters

In [81]:
# set number of clusters
kclusters = 7

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([6, 6, 5, 1, 1, 6, 6, 6, 6, 6], dtype=int32)

#### 8.2 Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

<h3><i> Optional Step since there was an index mismatch for my two arrays, I checked the difference and remove the addditional rows</i></h3>

In [82]:
n1=toronto_df_geo['Neighborhood'].tolist()
n2=neighborhoods_venues_sorted['Neighborhood'].tolist()
result=set(n1).difference(n2)
print(result)
print(n1.index('Islington Avenue'))
print(n1.index('Newtonbrook, Willowdale'))
print(n1.index('Upper Rouge'))


{'Islington Avenue', 'Newtonbrook, Willowdale', 'Upper Rouge'}
93
21
16


<h3></i> Drop the rows </i> </h3>

In [83]:
toronto_new_df=toronto_df_geo.drop(toronto_df_geo.index[[93,21,16]])


In [84]:

toronto_merged=toronto_new_df
# add clustering labels

toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,6,Fast Food Restaurant,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,College Gym
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,6,Bar,Women's Store,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,5,Rental Car Location,Medical Center,Electronics Store,Spa,Mexican Restaurant,Pizza Place,Breakfast Spot,Diner,Discount Store,Dog Run
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1,Coffee Shop,Korean Restaurant,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Women's Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1,Hakka Restaurant,Bakery,Fried Chicken Joint,Thai Restaurant,Caribbean Restaurant,Bank,Athletics & Sports,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


### 8.3 Visualize the resulting clusters

In [85]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 9. Examine Clusters

### Cluster 1

In [94]:
pizza_cluster=toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
pizza_cluster.loc[:,'Cluster Labels'] = 'Pizza'
pizza_cluster

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
99,Etobicoke,Pizza,Pizza Place,Coffee Shop,Middle Eastern Restaurant,Sandwich Place,Intersection,Chinese Restaurant,Women's Store,Doner Restaurant,Diner,Discount Store


In [96]:
restaurant_cluster=toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
restaurant_cluster.loc[:,'Cluster Labels'] = 'restaurant'
restaurant_cluster

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Scarborough,restaurant,Coffee Shop,Korean Restaurant,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Women's Store
4,Scarborough,restaurant,Hakka Restaurant,Bakery,Fried Chicken Joint,Thai Restaurant,Caribbean Restaurant,Bank,Athletics & Sports,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
10,Scarborough,restaurant,Indian Restaurant,Pet Store,Vietnamese Restaurant,Light Rail Station,Latin American Restaurant,Chinese Restaurant,Donut Shop,Dim Sum Restaurant,Diner,Discount Store
26,North York,restaurant,Japanese Restaurant,Gym / Fitness Center,Café,Basketball Court,Caribbean Restaurant,Baseball Field,Pool,Donut Shop,Discount Store,Dog Run
58,Downtown Toronto,restaurant,Coffee Shop,Café,American Restaurant,Steakhouse,Thai Restaurant,Restaurant,Bakery,Asian Restaurant,Gym,Hotel
60,Downtown Toronto,restaurant,Coffee Shop,Hotel,Café,American Restaurant,Restaurant,Gastropub,Italian Restaurant,Deli / Bodega,Gym,Sports Bar
62,North York,restaurant,Italian Restaurant,Coffee Shop,Juice Bar,Fast Food Restaurant,Pharmacy,Restaurant,Pub,Butcher,Café,Pizza Place
78,West Toronto,restaurant,Coffee Shop,Café,Breakfast Spot,Caribbean Restaurant,Furniture / Home Store,Burrito Place,Italian Restaurant,Climbing Gym,Convenience Store,Gym
90,Etobicoke,restaurant,Park,Pool,River,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Deli / Bodega
94,Etobicoke,restaurant,Bank,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Women's Store,Dim Sum Restaurant


In [98]:
shopping_cluster=toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
shopping_cluster.loc[:,'Cluster Labels'] = 'shopping'
shopping_cluster

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
54,Downtown Toronto,shopping,Coffee Shop,Clothing Store,Café,Middle Eastern Restaurant,Cosmetics Shop,Thai Restaurant,Restaurant,Ramen Restaurant,Plaza,Diner


In [99]:
food_cluster=toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
food_cluster.loc[:,'Cluster Labels'] = 'food'
food_cluster

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,North York,food,Coffee Shop,Pizza Place,Fast Food Restaurant,Sushi Restaurant,Restaurant,Shopping Mall,Fried Chicken Joint,Frozen Yogurt Shop,Bank,Diner


In [100]:
coffee_cluster=toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
coffee_cluster.loc[:,'Cluster Labels'] = 'coffee'
coffee_cluster

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
43,East Toronto,coffee,Café,Coffee Shop,Yoga Studio,Bakery,Italian Restaurant,American Restaurant,New American Restaurant,Bookstore,Seafood Restaurant,Sandwich Place
57,Downtown Toronto,coffee,Coffee Shop,Café,Italian Restaurant,Burger Joint,Ice Cream Shop,Bar,Chinese Restaurant,Middle Eastern Restaurant,Thai Restaurant,Spa


In [101]:
downtown_cluster=toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
downtown_cluster.loc[:,'Cluster Labels'] = 'downtown'
downtown_cluster

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,downtown,Rental Car Location,Medical Center,Electronics Store,Spa,Mexican Restaurant,Pizza Place,Breakfast Spot,Diner,Discount Store,Dog Run
13,Scarborough,downtown,Pizza Place,Chinese Restaurant,Italian Restaurant,Thai Restaurant,Noodle House,Fried Chicken Joint,Pharmacy,Fast Food Restaurant,Eastern European Restaurant,Dumpling Restaurant
17,North York,downtown,Mediterranean Restaurant,Dog Run,Golf Course,Pool,Drugstore,Diner,Discount Store,Doner Restaurant,Donut Shop,Women's Store
42,East Toronto,downtown,Park,Movie Theater,Sushi Restaurant,Steakhouse,Ice Cream Shop,Italian Restaurant,Burrito Place,Fish & Chips Shop,Burger Joint,Intersection
47,Central Toronto,downtown,Sandwich Place,Dessert Shop,Pizza Place,Seafood Restaurant,Café,Italian Restaurant,Restaurant,Coffee Shop,Sushi Restaurant,Pharmacy
48,Central Toronto,downtown,Playground,Restaurant,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
59,Downtown Toronto,downtown,Coffee Shop,Hotel,Aquarium,Café,Pizza Place,Brewery,Bakery,Scenic Lookout,Italian Restaurant,Restaurant
64,Central Toronto,downtown,Bus Line,Trail,Park,Sushi Restaurant,Jewelry Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
67,Downtown Toronto,downtown,Bar,Café,Vegetarian / Vegan Restaurant,Bakery,Vietnamese Restaurant,Coffee Shop,Dumpling Restaurant,Chinese Restaurant,Mexican Restaurant,Caribbean Restaurant
74,York,downtown,Park,Fast Food Restaurant,Pharmacy,Market,Women's Store,Gourmet Shop,Golf Course,Grocery Store,Electronics Store,Eastern European Restaurant


<h2>*********************************** THANK YOU **************************************</h2>